# Understanding Ontologies with Cognee

This notebook demonstrates how to work with ontologies in scientific research using the Cognee framework. We'll explore how ontologies can enhance our understanding and querying of scientific papers.

## What is an Ontology?

An ontology is a formal representation of knowledge that defines:
- Concepts within a domain
- Relationships between concepts
- Properties and attributes
- Rules and constraints

Key terms:
- **Classes**: Categories or types (e.g., Disease, Symptom)
- **Instances**: Specific examples of classes (e.g., Type 2 Diabetes)
- **Properties**: Relationships between classes/instances (e.g., hasSymptom)
- **Axioms**: Logical statements defining relationships

## Setup

First, let's install the required packages and set up our environment:

In [1]:
# Install required package
# !pip install cognee

In [2]:
# Import required libraries
import os
import cognee
from cognee.shared.logging_utils import get_logger
from cognee.api.v1.search import SearchType

logger = get_logger()

# Set up OpenAI API key (required for Cognee's LLM functionality)
os.environ["LLM_API_KEY"] = "your-api-key-here"  # Replace with your API key


2025-06-18T18:23:32.523592 [info     ] Deleted old log file: /Users/borisarzentar/Projects/Topoteretes/cognee/logs/2025-06-18_20-08-11.log [cognee.shared.logging_utils]

2025-06-18T18:23:32.524072 [info     ] Logging initialized            [cognee.shared.logging_utils] cognee_version=0.1.42-dev os_info='Darwin 24.5.0 (Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:25 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T6020)' python_version=3.11.5 structlog_version=25.4.0

HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/borisarzentar/Projects/Topoteretes/cognee/.venv/lib/python3.11/site-packages/dlt/helpers/dbt/__init__.

## Creating the Pipeline

Let's create a pipeline that will:
1. Clean existing data
2. Process scientific papers
3. Apply ontological knowledge

In [3]:
async def run_pipeline(ontology_path=None):
    # Clean existing data
    await cognee.prune.prune_data()
    await cognee.prune.prune_system(metadata=True)
    
    # Set up path to scientific papers
    scientific_papers_dir = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath("."))), 
        "cognee",
        "examples",
        "data", 
        "scientific_papers/"
    )
    
    # Add papers to the system
    await cognee.add(scientific_papers_dir)
    
    # Cognify with optional ontology
    return await cognee.cognify(ontology_file_path=ontology_path)

async def query_pipeline(questions):
    answers = []
    for question in questions:
        search_results = await cognee.search(
            query_type=SearchType.GRAPH_COMPLETION,
            query_text=question,
        )
        answers.append(search_results)
    return answers

## Running the Demo

Let's test our system with some medical questions, comparing results with and without ontological knowledge:

In [4]:
# Test questions
questions = [
    "What are common risk factors for Type 2 Diabetes?",
    "What preventive measures reduce the risk of Hypertension?",
    "What symptoms indicate possible Cardiovascular Disease?",
    "What diseases are associated with Obesity?"
]

# Path to medical ontology
ontology_path = "examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl"  # Update with your ontology path

# Run with ontology
print("\n--- Results WITH ontology ---\n")
await run_pipeline(ontology_path=ontology_path)
answers_with = await query_pipeline(questions)
for q, a in zip(questions, answers_with):
    print(f"Q: {q}\nA: {a}\n")


2025-06-18T18:23:36.293948 [info     ] Cleared all data from graph while preserving structure [cognee.shared.logging_utils]
2025-06-18T18:23:36.358529 [info     ] Database deleted successfully. [cognee.shared.logging_utils]


--- Results WITH ontology ---




Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client20:23:36 - LiteLLM:INFO: utils.py:3101 - 
LiteLLM completion() model= gpt-5-mini; provider = openai

LiteLLM completion() model= gpt-5-mini; provider = openai20:23:37 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-5-mini-2024-07-18
selected model name for cost calculation: openai/gpt-5-mini-2024-07-18
EmbeddingRateLimiter initialized: enabled=False, requests_limit=60, interval_seconds=6020:23:37 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-5-mini-2024-07-18
selected model name for cost calculation: openai/gpt-5-mini-2024-07-1820:23:37 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/text-embedding-3-large
selected model name for cost calculation: op

User c0d22401-0c0b-40ad-8ce5-8d6094b0461d has registered.



2025-06-18T18:23:38.501726 [info     ] Coroutine task completed: `ingest_data` [run_tasks_base]
2025-06-18T18:23:38.502195 [info     ] Coroutine task completed: `resolve_data_directories` [run_tasks_base]
2025-06-18T18:23:38.502710 [info     ] Pipeline run completed: `2bec40b8-e3d1-54ab-bfc5-eb5d4695ce63` [run_tasks(tasks: [Task], data)]
2025-06-18T18:23:38.504289 [info     ] Ontology file 'examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl' not found. No owl ontology will be attached to the graph. [OntologyAdapter]
2025-06-18T18:23:38.513310 [info     ] Pipeline run started: `ba0e59ba-8966-58b2-8dbe-0d3d5009b268` [run_tasks(tasks: [Task], data)]
2025-06-18T18:23:38.513615 [info     ] Coroutine task started: `classify_documents` [run_tasks_base]
2025-06-18T18:23:38.513906 [info     ] Coroutine task started: `check_permissions_on_dataset` [run_tasks_base]
2025-06-18T18:23:38.517146 [info     ] Async Generator task started: `extract_chunks_from_documents` 

Q: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes include:\n- Obesity (Body Mass Index ≥30 kg/m²)\n- High waist circumference (increased risk category)\n- Sedentary lifestyle (low physical activity)\n- Smoking status (current smokers)\n- Hypertension\n- High cholesterol\n- Poor nutrition (related to coffee consumption)\n']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures that reduce the risk of hypertension include moderate coffee consumption, which is associated with a lower risk of developing hypertension. This effect is more pronounced in individuals who are non-smokers or fast caffeine metabolizers. Additionally, filtered coffee is recommended over boiled coffee due to its antiatherogenic properties and lower cholesterol impact.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Possible symptoms indicating cardiovascular disease include hypertension, heart failure, and coronary he

In [5]:
# Run without ontology
print("\n--- Results WITHOUT ontology ---\n")
await run_pipeline()
answers_without = await query_pipeline(questions)
for q, a in zip(questions, answers_without):
    print(f"Q: {q}\nA: {a}\n")


2025-06-18T18:24:33.294076 [info     ] Cleared all data from graph while preserving structure [cognee.shared.logging_utils]
2025-06-18T18:24:33.317640 [info     ] Database deleted successfully. [cognee.shared.logging_utils]
2025-06-18T18:24:33.387322 [info     ] Pipeline run started: `0693bdfd-667e-5f24-adf4-81dc64b99cb4` [run_tasks(tasks: [Task], data)]
2025-06-18T18:24:33.387792 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]
2025-06-18T18:24:33.388288 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]


--- Results WITHOUT ontology ---

User 2da365d6-bd7c-4750-807e-74e1f340d5d2 has registered.



2025-06-18T18:24:33.505418 [info     ] Coroutine task completed: `ingest_data` [run_tasks_base]
2025-06-18T18:24:33.505934 [info     ] Coroutine task completed: `resolve_data_directories` [run_tasks_base]
2025-06-18T18:24:33.506308 [info     ] Pipeline run completed: `0693bdfd-667e-5f24-adf4-81dc64b99cb4` [run_tasks(tasks: [Task], data)]
2025-06-18T18:24:33.507654 [info     ] Ontology file 'None' not found. No owl ontology will be attached to the graph. [OntologyAdapter]
2025-06-18T18:24:33.515544 [info     ] Pipeline run started: `63d064a5-2884-5c10-9aeb-38e16d5955ea` [run_tasks(tasks: [Task], data)]
2025-06-18T18:24:33.515829 [info     ] Coroutine task started: `classify_documents` [run_tasks_base]
2025-06-18T18:24:33.516136 [info     ] Coroutine task started: `check_permissions_on_dataset` [run_tasks_base]
2025-06-18T18:24:33.519493 [info     ] Async Generator task started: `extract_chunks_from_documents` [run_tasks_base]
2025-06-18T18:24:33.807889 [info     ] Coroutine task starte

Q: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes include:\n1. Obesity (high body mass index)\n2. High cholesterol levels\n3. Sedentary lifestyle (low physical activity)\n4. Poor diet choices, particularly low adherence to healthy diets like the Mediterranean diet\n5. Age (usually occurs in adults)\n6. Family history of diabetes\n7. Smoking and alcohol consumption.']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures to reduce the risk of hypertension include moderate coffee consumption, which is linked to lower hypertension, improved cardiovascular health, and reduced incidence of atrial fibrillation. Additionally, antioxidants found in foods can also have protective effects.']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Possible symptoms that may indicate cardiovascular disease include hypertension (consistently elevated blood pressure) and heart failure (a chronic condition in wh

## Visualizing the Knowledge Graph

Let's visualize how our ontology connects different medical concepts:

In [6]:
from cognee.api.v1.visualize import visualize_graph
await visualize_graph()


2025-06-18T18:25:30.641928 [info     ] Graph visualization saved as /Users/borisarzentar/graph_visualization.html [cognee.shared.logging_utils]
2025-06-18T18:25:30.642469 [info     ] The HTML file has been stored on your home directory! Navigate there with cd ~ [cognee.shared.logging_utils]

'\n    <!DOCTYPE html>\n    <html>\n    <head>\n        <meta charset="utf-8">\n        <script src="https://d3js.org/d3.v5.min.js"></script>\n        <style>\n            body, html { margin: 0; padding: 0; width: 100%; height: 100%; overflow: hidden; background: linear-gradient(90deg, #101010, #1a1a2e); color: white; font-family: \'Inter\', sans-serif; }\n\n            svg { width: 100vw; height: 100vh; display: block; }\n            .links line { stroke: rgba(255, 255, 255, 0.4); stroke-width: 2px; }\n            .nodes circle { stroke: white; stroke-width: 0.5px; filter: drop-shadow(0 0 5px rgba(255,255,255,0.3)); }\n            .node-label { font-size: 5px; font-weight: bold; fill: white; text-anchor: middle; dominant-baseline: middle; font-family: \'Inter\', sans-serif; pointer-events: none; }\n            .edge-label { font-size: 3px; fill: rgba(255, 255, 255, 0.7); text-anchor: middle; dominant-baseline: middle; font-family: \'Inter\', sans-serif; pointer-events: none; }\n     

## Understanding the Results

The demonstration above shows how ontologies enhance our analysis by:

1. **Making Connections**: 
   - Linking related medical concepts even when not explicitly stated
   - Identifying relationships between symptoms, diseases, and risk factors

2. **Standardizing Terms**: 
   - Unifying different ways of referring to the same medical condition
   - Ensuring consistent terminology across documents

3. **Enabling Inference**: 
   - Drawing conclusions based on ontological relationships
   - Discovering implicit connections in the data

## Next Steps

To learn more about Cognee and ontologies:
1. Check out the [Cognee documentation](https://docs.cognee.ai/)
2. Explore more examples in the `examples` directory
3. Try creating your own domain-specific ontology

Remember to:
- Place your scientific papers in the appropriate directory
- Update the ontology path to point to your .owl file
- Replace the API key with your own OpenAI key